In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

trainDG = ImageDataGenerator(rescale = 1./255,
                        rotation_range=90,
                        width_shift_range=0.3,
                        height_shift_range=0.3,
                        shear_range=0.2,
                        zoom_range=0.3,
                        horizontal_flip=True
                        )

trainG = trainDG.flow_from_directory("C:\\Users\\ronit\\AI\\kaggle_data\\SkinCancerDataset(8c)\\train",
                                target_size = (190, 190),
                                class_mode = 'categorical',
                                shuffle = True
                                )
                                
valDG = ImageDataGenerator(rescale = 1./255,
                    rotation_range = 90,
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    shear_range = 0.2,
                    zoom_range = 0.3,
                    horizontal_flip = True
                    )
valG = valDG.flow_from_directory("C:\\Users\\ronit\\AI\\kaggle_data\\SkinCancerDataset(8c)\\validation",
                            target_size = (190, 190),
                            class_mode = 'categorical',
                            shuffle = True)

Found 18974 images belonging to 8 classes.
Found 3794 images belonging to 8 classes.


In [9]:
from hyperopt import hp

space = dict(bs=hp.randint('bs', 64, 128),
            fil1=hp.randint('fil1', 1024),
            fil2=hp.randint('fil2', 1024),
            fil3=hp.randint('fil3', 1024),
            fil4=hp.randint('fil4', 1024),
            win1=hp.choice('win1', [3, 5, 7]),
            win2=hp.choice('win2', [3, 5, 7]),
            win3=hp.choice('win3', [3, 5, 7]),
            win4=hp.choice('win4', [3, 5, 7])
            )
print(space['bs'])

0 hyperopt_param
1   Literal{bs}
2   randint
3     Literal{64}
4     Literal{128}


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, AveragePooling2D, BatchNormalization

def objective(bs, fil1, fil2, fil3, fil4, win1, win2, win3, win4):
    model = Sequential()
    '''
    model.add(Conv2D(fil1, (win1, win1), activation='relu', input_shape = (190, 190, 3), data_format = "channels_last"))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(fil2, (win2, win2), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(fil3, (win3, win3), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Conv2D(fil4, (win4, win4), activation='relu'))
    model.add(MaxPool2D((2,2)))

    model.add(Flatten())

    model.add(Dropout(0.4))

    model.add(Dense(128, activation='relu'))
    '''    
    model.add(Dense(8, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
    from datetime import datetime

    estop = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=17)


    log_dir = 'C:\\Users\\ronit\\AI\\kaggle_ml\\tensorboard'
    timestamp = datetime.now().strftime("%Y-%m-%d--%H%M"+'_(skin-cancer-model8c)')
    log_dir = log_dir + '\\' + timestamp

    tboard = TensorBoard(log_dir=log_dir, 
            histogram_freq=1,
            write_graph=True,
            write_images=True,
            update_freq='epoch',
            profile_batch=2,
            embeddings_freq=1)

    cpoint = ModelCheckpoint(filepath = 'C:\\Users\\ronit\\AI\\kaggle_ml\\skin_cancer_model8c_{val_accuracy: .3f}val_acc.h5', 
                save_best_only = True, 
                save_weights_only = False, 
                monitor = 'val_accuracy',
                mode = 'max')
    history = model.fit(trainG, 
            epochs=40, 
            steps_per_epoch=int(18974/bs), 
            validation_data = valG, 
            verbose = 1, 
            validation_steps=int(3794/bs),
            callbacks = [estop, tboard, cpoint],
            batch_size = bs
            )

    return history.history['accuracy'][len(history.history['accuracy'])-1]

In [6]:
'''from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import random'''
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import tpe, Trials, fmin
def f(x):
    return x*2
tpeAlg = tpe.suggest
tpeTri = Trials()
tpeMin = fmin(fn = f, space = space, algo = tpeAlg, trials = tpeTri, max_evals = 100)

0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]job exception: unsupported operand type(s) for *: 'dict' and 'int'

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


TypeError: unsupported operand type(s) for *: 'dict' and 'int'